<a href="https://colab.research.google.com/github/pgordin/Grafy2022/blob/main/Grafy2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importowanie pakietów

In [1]:
import numpy as np
from random import random, seed

# Funkcje grafowe

In [2]:
def print_matrix(vertices, matrix):
  """
  Wypisuje na ekreanie graf zadany jako macierz sąsiedztwa
  """
  n = len(matrix)
  if (vertices is not None) and (len(vertices) == n):
    vv = vertices
  else:
    vv = range(1, n+1)
  for i in range(n):
    print(vv[i], ":", end="")
    for j in range(n):
      if matrix[i,j]:
        print(" ", vv[j], end="")
    print("")

In [3]:
def print_graph(graph):
  """
  Wypisuje graf zadany jako słownik pythona
  """
  for v in graph:
    print(v, ":", end="")
    for u in graph[v]:
      print(" ", u, end="")
    print("")
    

## Tworzenie grafu (jako listy sąsiedztwa)

In [4]:
def add_vertex(graph, vertex):
    """Nowy wierzchołek do istniejącego grafu"""
    if vertex not in graph:
        graph[vertex] = []


def add_arc(graph, arc):
    """Dodaje nowy łuk (parę wierzchołków) do istniejącego grafu
       rozważamy grafy proste, skierowane
    """
    u, v = arc
    add_vertex(graph, u)
    add_vertex(graph, v)
    if v not in graph[u]:
        graph[u].append(v)


def add_edge(graph, edge):
    """Dodaje nową krawędź (parę wierzchołków) do istniejącego grafu
       traktując graf nieskierowany prosty jako prosty graf skierowany, ale symetryczny i bez pętli
    """
    u, v = edge
    add_vertex(graph, u)
    add_vertex(graph, v)
    if u == v:
        raise ValueError("pętla!")
    if v not in graph[u]:
        graph[u].append(v)
    if u not in graph[v]:
        graph[v].append(u)

# Praca domowa nr 1

In [7]:
def random_graph(n,p):
  if 0<=p and p<=1:
    random_graph={}
    for i in range(1,n+1):
      add_vertex(random_graph, i)
      for j in range(1,i):
        if random() < p:
          add_edge(random_graph, [i,j])
  else:
    raise ValueError("To jest graf losowy")
  return random_graph

def graph_to_matrix(graph):
  n=len(graph)
  macierz_sasiedztwa=np.zeros((n,n)) #to jest macierz zerowa n x n
  wiersz = 0
  for i in graph: #to przechodzi po każdym wierzchołu
    for j in graph.get(i):#bierze liste sąsiadów tego grafu
     macierz_sasiedztwa[wiersz][j-1] += 1
    wiersz +=1
  return macierz_sasiedztwa

def matrix_to_graph(wektor,macierz):
  n=len(wektor)
  graph={}
  if (n == len(macierz)) and (wektor is not None):
    for i in range(0,n):
      add_vertex(graph,wektor[i])
    for i in range(0,n):
      for j in range(1,n):
        if macierz[i][j] == 1:
          add_edge(graph,[wektor[i],wektor[j]])
  else:
    raise ValueError("Coś źle podałeś")
  print(graph)


"""
graph1 = random_graph(10, 1/3)
macierz = graph_to_matrix(graph1)
print_graph(graph1)
print("-----------------")
print_matrix(None, macierz)
"""

1 :  3  7  9
2 :  6
3 :  1  7  9
4 :  5  9
5 :  4  9
6 :  2
7 :  1  3  10
8 :
9 :  1  3  4  5
10 :  7
-----------------
1 :  3  7  9
2 :  6
3 :  1  7  9
4 :  5  9
5 :  4  9
6 :  2
7 :  1  3  10
8 :
9 :  1  3  4  5
10 :  7


# Wczytywanie (i zapis) grafów z plików

In [13]:
def graph_from_edges(filename, directed = 0):
  """
  Wczytuje graf z pliku tekstowego, w któym w każdej linii jest opis jednej krawędzi (pary słów),
  ewentualnie jednego wierzchołka (pojedyncze słowo). Jako wynik zwraca graf w formie listy sąsiedztwa.
  Plik musi być w bierzącym katalogu lub filename zawiera całą ścieżkę do pliku
  """
  graph = {}
  file = open(filename, "r")  # otwarcie pliku do odczytu
  for line in file:           # dla każdej liniii w pliku
    words = line.split()      # dzielę linię na słowa (wg spacji)
    if len(words) == 1:       # jedno słowo - wierzchołek
      add_vertex(graph, words[0])
    elif len(words) >= 2:     # więcej słów - używam dwóch pierwszych
      if directed:
        add_arc(graph, (words[0], words[1]))
      else:
        add_edge(graph, (words[0], words[1]))
  file.close()
  return graph

In [17]:
def graph_to_neigbourlist(graph, filename):
  """
  Zapisuje graf jako listę sąsiedztwa do pliku tekstowego filename.
  """
  file = open(filename, "w")  # otwarcie pliku do zapisu
  for v in graph:
    neigh_list = "{}:".format(v)  # buduję napis (przez format) - na razie 'v:'
    for u in graph[v]:
      neigh_list = neigh_list + " {}".format(u) # dołączamy ' u' na koniec napisu
    neigh_list += '\n'            # koniec wiersza
    file.write(neigh_list)        # zapisujemy wiersz do pliku
  file.close()

# Kody Prüfera

In [28]:
from copy import deepcopy

def Prufer(tree):
  """
  Kod Prüfera dla drzewa - podany jako napis
  """
  tr = deepcopy(tree)               # będziemy psuć drzewo
  code = ""
  for i in range(len(tree) - 2):
    for x in sorted(tr):  # po kolei przeglądam wierzchołki - od najmniejszego
      if len(tr[x]) == 1: # liść - pierszy - najmniejszy
        break
    v = tr[x][0]          # sąsiad najmniejszego liścia
    code = code + " {}".format(v)
    tr[v].remove(x)       # usuwam x z listy sąsiadów v
    tr.pop(x)             # usuwam x z drzewa
  return code


def TreeFromPrufer(code):
  """
  Tworzenie drzewa z kodu Prüfera - kod jest napisem długości n-2 (z liczb od 1 do n)
  drzewo - grafem w formie lsity sąsiadów.
  """
  clist = [int(x) for x in code.split()]  # kod zamieniony na listę liczb
  n = len(clist) + 2                      # liczba wierzchołków
  vert = [v for v in range(1, n+1)]       # lista wszystkich wierzchołków drzewa - 1..n
  tree = {}
  for v in vert:
    add_vertex(tree, v)
  for i in range(n-2):
    for x in vert:
      if x not in clist:  # x - najmniejszy liść (na tym etapie)
        break
    v = clist.pop(0)  # pierwszy element listy/kodu
    add_edge(tree, (x, v))
    vert.remove(x)
  add_edge(tree, (vert[0], vert[1]))
  return tree

# Wykorzystanie kodu (1)

### Pierwszy przykładowy graf.

In [ ]:
vertices = ["a", "b", "c", "d", "e", "f"]
matrix = np.array([[0,1,1,0,0,0],[1,0,1,0,0,0],[1,1,0,1,1,0],[0,0,1,0,1,0],[0,0,1,1,0,0],[0,0,0,0,0,0]])
print(vertices)
print(matrix)
print("-------------------")
print_matrix(vertices, matrix)
print("-------------------")
print_matrix(None, matrix)

['a', 'b', 'c', 'd', 'e', 'f']
[[0 1 1 0 0 0]
 [1 0 1 0 0 0]
 [1 1 0 1 1 0]
 [0 0 1 0 1 0]
 [0 0 1 1 0 0]
 [0 0 0 0 0 0]]
-------------------
a :  b  c
b :  a  c
c :  a  b  d  e
d :  c  e
e :  c  d
f :
-------------------
1 :  2  3
2 :  1  3
3 :  1  2  4  5
4 :  3  5
5 :  3  4
6 :


### Druga reprezentacja

In [ ]:
graph = {
  'a' : ['b', 'c'],
  'b' : ['a', 'c'],
  'c' : ['a', 'b', 'd', 'e'],
  'd' : ['c', 'e'],
  'e' : ['c', 'd'],
  'f' : []
}
print(graph)
print("-------------------")
print_graph(graph)

{'a': ['b', 'c'], 'b': ['a', 'c'], 'c': ['a', 'b', 'd', 'e'], 'd': ['c', 'e'], 'e': ['c', 'd'], 'f': []}
-------------------
a :  b  c
b :  a  c
c :  a  b  d  e
d :  c  e
e :  c  d
f :


### Modyfikacje grafów

In [ ]:
add_vertex(graph, "g")
add_arc(graph, ['a', 'g'])
add_arc(graph, ['g', 'a'])
add_edge(graph, ['a', 'h'])
add_edge(graph, ['h', 'd'])
#błąd add_edge(graph, ['h', 'h'])
add_arc(graph, ['h', 'h'])
print_graph(graph)


a :  b  c  g  h
b :  a  c
c :  a  b  d  e
d :  c  e  h
e :  c  d
f :
g :  a
h :  a  d  h


### Tworzenie grafów losowych $ G(n, p) $

In [ ]:
# dla powtarzalności wyników
seed(2021)

In [ ]:
#losuję graf G(10, 1/3)
n = 10
p = 1/3
random_graph = {}
for i in range(1, n+1):
  add_vertex(random_graph, i)
  for j in range(1, i):
    if random() < p:
      add_edge(random_graph, [i, j])

print_graph(random_graph)

1 :  4  9  10
2 :  5  6  7  9  10
3 :  5
4 :  1  8
5 :  2  3  7  9  10
6 :  2  10
7 :  2  5
8 :  4  9
9 :  1  2  5  8
10 :  1  2  5  6


# Wykorzystanie kodu (2)

In [10]:
%ls

lista.txt  sample_data/


In [9]:
%%writefile lista.txt
1
2 3
2 4
2 5
3 5
6
1 2

Writing lista.txt


In [11]:
%cat lista.txt

1
2 3
2 4
2 5
3 5
6
1 2


In [14]:
graph2 = graph_from_edges("lista.txt", 0)
print_graph(graph2)

1 :  2
2 :  3  4  5  1
3 :  2  5
4 :  2
5 :  2  3
6 :


In [15]:
graph3 = graph_from_edges("ubranie.txt", directed=1)
print_graph(graph3)

slipki :  kalesony
kalesony :  spodnie
spodnie :  buty  szelki
buty :
szelki :  marynarka
skarpety :  buty
koszula :  szelki  marynarka  krawat
marynarka :  plaszcz
krawat :  marynarka
plaszcz :


In [18]:
graph_to_neigbourlist(graph2, "graf_l.txt")
graph_to_neigbourlist(graph3, "ubranie_l.txt")
%cat "graf_l.txt"
print("--------------------")
%cat "ubranie_l.txt"

1: 2
2: 3 4 5 1
3: 2 5
4: 2
5: 2 3
6:
--------------------
slipki: kalesony
kalesony: spodnie
spodnie: buty szelki
buty:
szelki: marynarka
skarpety: buty
koszula: szelki marynarka krawat
marynarka: plaszcz
krawat: marynarka
plaszcz:


In [19]:
%%writefile drzewo1.txt
1
2 3
2 4
2 5
5 6
1 2

Writing drzewo1.txt


In [31]:
drzewo1 = graph_from_edges("drzewo1.txt", 0)
print_graph(drzewo1)
print(Prufer(drzewo1))
print_graph(drzewo1)
print("----------------")
print_graph(TreeFromPrufer("2 2 2 5"))
print("----------------")
print_graph(TreeFromPrufer("2 2 2 2 2 2 2 2"))
print("----------------")
print_graph(TreeFromPrufer("2 3 4 5 6 7 8 9"))



1 :  2
2 :  3  4  5  1
3 :  2
4 :  2
5 :  2  6
6 :  5
 2 2 2 5
1 :  2
2 :  3  4  5  1
3 :  2
4 :  2
5 :  2  6
6 :  5
----------------
1 :  2
2 :  1  3  4  5
3 :  2
4 :  2
5 :  2  6
6 :  5
----------------
1 :  2
2 :  1  3  4  5  6  7  8  9  10
3 :  2
4 :  2
5 :  2
6 :  2
7 :  2
8 :  2
9 :  2
10 :  2
----------------
1 :  2
2 :  1  3
3 :  2  4
4 :  3  5
5 :  4  6
6 :  5  7
7 :  6  8
8 :  7  9
9 :  8  10
10 :  9
